In [7]:
# DB/database.py
import sqlite3

# Conectar ao banco de dados SQLite
conn = sqlite3.connect('loteria.db')
c = conn.cursor()

# Criar tabela de Usuários
c.execute('''CREATE TABLE IF NOT EXISTS usuarios (
             id INTEGER PRIMARY KEY,
             username TEXT NOT NULL,
             password TEXT NOT NULL
             )''')

# Criar tabela de Jogos
c.execute('''CREATE TABLE IF NOT EXISTS jogos (
             id INTEGER PRIMARY KEY,
             usuario_id INTEGER NOT NULL,
             numeros_jogados TEXT,
             resultado INTEGER,
             FOREIGN KEY (usuario_id) REFERENCES usuarios (id)
             )''')

# Criar tabela de Backlog (para registro de dados de simulação)
c.execute('''CREATE TABLE IF NOT EXISTS backlog (
             id INTEGER PRIMARY KEY,
             usuario_id INTEGER NOT NULL,
             numeros_simulados TEXT,
             data_simulacao TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
             FOREIGN KEY (usuario_id) REFERENCES usuarios (id)
             )''')

# Commit e fechar conexão
conn.commit()
conn.close()


In [8]:
import sqlite3

# Criar conexão com o banco de dados
conn = sqlite3.connect('database.db')
c = conn.cursor()

# Verificar campos da tabela Usuarios
c.execute("PRAGMA table_info(Usuarios)")
print("Campos da tabela Usuarios:")
for row in c.fetchall():
    print(f"  {row[1]} ({row[2]})")

# Verificar registros da tabela Usuarios
c.execute("SELECT * FROM Usuarios")
print("\nRegistros da tabela Usuarios:")
for row in c.fetchall():
    print(f"  {row}")

# Verificar campos da tabela Jogos
c.execute("PRAGMA table_info(Jogos)")
print("\nCampos da tabela Jogos:")
for row in c.fetchall():
    print(f"  {row[1]} ({row[2]})")

# Verificar registros da tabela Jogos
c.execute("SELECT * FROM Jogos")
print("\nRegistros da tabela Jogos:")
for row in c.fetchall():
    print(f"  {row}")

# Fechar a conexão
conn.close()

Campos da tabela Usuarios:
  id (INTEGER)
  nome (TEXT)
  email (TEXT)
  senha (TEXT)

Registros da tabela Usuarios:

Campos da tabela Jogos:
  id (INTEGER)
  usuario_id (INTEGER)
  jogo (TEXT)
  data_criacao (TEXT)

Registros da tabela Jogos:
